In [1]:
import numpy as np, trimesh as tm
from pathlib import Path

TARGET_SIZE = np.array([30., 30., 36.])      # mm  (X, Y, Z)
PLATE_THICK = 3.0                            # mm  (each end)

def preprocess_mesh(stl_path: Path) -> tm.Trimesh:
    """Load STL, scale so height=36 mm, remove 3 mm plates, return new mesh."""
    mesh = tm.load_mesh(stl_path, process=False)

    # --- 1️⃣ scale: map longest axis → 36 mm -------------------------------
    extents = mesh.bounding_box_oriented.extents  # (dx,dy,dz)
    tallest_axis = extents.argmax()               # 0,1,2
    scale = TARGET_SIZE[2] / extents[tallest_axis]
    mesh.apply_scale(scale)

    # --- 2️⃣ rotate so Z is the tall axis, if necessary --------------------
    if tallest_axis != 2:
        # permutation matrices for X→Z, Y→Z
        rot = {0: tm.transformations.rotation_matrix(np.pi/2, [0,1,0]),
               1: tm.transformations.rotation_matrix(-np.pi/2, [1,0,0])}[tallest_axis]
        mesh.apply_transform(rot)

    # --- 3️⃣ slice off plates ---------------------------------------------
    z_min, z_max = mesh.bounds[:,2]
    planes = [
        ( [0,0, 1], -(z_min + PLATE_THICK) ),    # bottom cut
        ( [0,0,-1], +(z_max - PLATE_THICK) )     # top cut
    ]
    for normal, offset in planes:
        mesh = mesh.slice_plane(plane_origin=[0,0, -offset*normal[2]],
                                plane_normal=normal)

    mesh.remove_unreferenced_vertices()
    return mesh


In [6]:
import numpy as np, trimesh as tm

def box_count_fractal_dimension(mesh: tm.Trimesh,
                                min_div=1, max_div=6):
    """
    Box-count fractal dimension via trimesh.voxel.
    • min_div=1 → largest box edge  L/2¹
    • max_div=6 → smallest box edge L/2⁶
    Returns (scales_mm, counts, D).
    """
    L = mesh.bounding_box.extents.max()     # mm, size of enclosing cube
    scales, counts = [], []

    for k in range(min_div, max_div + 1):
        pitch = L / (2 ** k)                # cube edge length ε
        vg = mesh.voxelized(pitch)          # trimesh.VoxelGrid
        scales.append(pitch)
        counts.append(len(vg.sparse_indices))

    coeff = np.polyfit(np.log(scales), np.log(counts), 1)
    D = -coeff[0]
    return np.array(scales), np.array(counts), D


In [7]:
def dimension_for_stl(path):
    mesh = preprocess_mesh(path)
    scales, counts, D = box_count_fractal_dimension(mesh, max_div=7)
    print(f"{path.name}:  D ≈ {D:.3f}")
    return scales, counts, D


In [8]:
from tqdm.auto import tqdm

def run_folder(folder):
    folder = Path(folder)
    for stl in tqdm(list(folder.rglob("*.stl")), desc=folder.name):
        dimension_for_stl(stl)

base = Path("3D_Files")
run_folder(base / "Grafted CTs")
run_folder(base / "Randomly Generated ICs")


Grafted CTs: 0it [00:00, ?it/s]

Randomly Generated ICs:   0%|          | 0/14 [00:00<?, ?it/s]

1.0578 1081.stl:  D ≈ 2.055
1.2000 1510.stl:  D ≈ 2.086
1.2268 1785.stl:  D ≈ 2.180
1.2799 2395.stl:  D ≈ 2.176
1.2799 3454.stl:  D ≈ 2.210
1.3156 2569.stl:  D ≈ 2.169
1.3156 3128.stl:  D ≈ 2.207
1.3156 3637.stl:  D ≈ 2.260
1.3244 2441.stl:  D ≈ 2.196
1.3244 4389.stl:  D ≈ 2.293
1.3423 4769.stl:  D ≈ 2.308
1.3423 6050.stl:  D ≈ 2.367
1.3512 5660.stl:  D ≈ 2.355
1.3689 5500.stl:  D ≈ 2.343


In [2]:
# ────────────────────────────────────────────────────────────────────────────
# 3-D Box-Counting   —   RANDOMLY-GENERATED CTs ONLY
# ────────────────────────────────────────────────────────────────────────────

from pathlib import Path
import subprocess, sys, importlib

# ---------- ensure required libs ----------
for pkg in ("trimesh", "pyembree", "numpy", "matplotlib", "tqdm"):
    if importlib.util.find_spec(pkg) is None:
        subprocess.check_call([sys.executable, "-m", "pip", "install", pkg, "--quiet"])

import numpy as np, trimesh as tm
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

# ---------- configuration ----------
SRC_ROOT      = Path("3D_Files") / "Randomly Generated ICs"   # ONLY this sub-dir
DST_STL_ROOT  = Path("Edited_STLs")      / "Randomly Generated ICs"
DST_PLOT_ROOT = Path("Boxcount_Plots")   / "Randomly Generated ICs"

TARGET_SIZE   = np.array([30., 30., 36.])   # mm  (X,Y,Z) after scaling
PLATE_THICK   = 3.0                         # mm slice off each end
VOXEL_DIV     = (1, 7)                      # ε = L/2^k  for k = 1 … 7

DST_STL_ROOT.mkdir(parents=True, exist_ok=True)
DST_PLOT_ROOT.mkdir(parents=True, exist_ok=True)

# ---------- helpers ----------
def preprocess_mesh(stl_path: Path) -> tm.Trimesh:
    mesh = tm.load_mesh(stl_path, process=False)

    # scale so tallest axis == 36 mm
    ext  = mesh.bounding_box_oriented.extents
    tall = ext.argmax()
    mesh.apply_scale(TARGET_SIZE[2] / ext[tall])

    # rotate so Z is the tall axis
    if tall != 2:
        rot = {0: tm.transformations.rotation_matrix(np.pi/2, [0,1,0]),
               1: tm.transformations.rotation_matrix(-np.pi/2, [1,0,0])}[tall]
        mesh.apply_transform(rot)

    # slice off loading plates
    z_min, z_max = mesh.bounds[:, 2]
    planes = ([0,0, 1], -(z_min + PLATE_THICK)), ([0,0,-1], +(z_max - PLATE_THICK))
    for n, off in planes:
        mesh = mesh.slice_plane([0,0,-off*n[2]], n)

    mesh.remove_unreferenced_vertices()
    return mesh

def box_count(mesh: tm.Trimesh, k_min, k_max):
    L = mesh.bounding_box.extents.max()
    scales, counts = [], []
    for k in range(k_min, k_max + 1):
        eps = L / (2**k)
        vox = mesh.voxelized(eps)
        scales.append(eps)
        counts.append(len(vox.sparse_indices))
    coeff = np.polyfit(np.log(scales), np.log(counts), 1)
    return np.array(scales), np.array(counts), -coeff[0], coeff

def plot_loglog(scales, counts, coeff, png_path, title=""):
    plt.figure(figsize=(4.6,4))
    plt.loglog(scales, counts, "o", label="data")
    plt.loglog(scales, np.exp(np.polyval(coeff, np.log(scales))),
               "-", label=f"fit  D={-coeff[0]:.3f}")
    plt.gca().invert_xaxis()
    plt.xlabel("box size ε [mm]")
    plt.ylabel("boxes N(ε)")
    plt.title(title);  plt.legend();  plt.tight_layout()
    png_path.parent.mkdir(parents=True, exist_ok=True)
    plt.savefig(png_path, dpi=300);  plt.close()

# ---------- main loop ----------
for stl in tqdm(list(SRC_ROOT.rglob("*.stl")), desc="Random ICs"):
    rel   = stl.relative_to(SRC_ROOT)
    print(f"\n→ {rel}")
    mesh  = preprocess_mesh(stl)

    # save cleaned STL
    out_stl = DST_STL_ROOT / rel
    out_stl.parent.mkdir(parents=True, exist_ok=True)
    mesh.export(out_stl)
    print(f"   cleaned STL  → {out_stl}")

    # box-count + plot
    scales, counts, D, coeff = box_count(mesh, *VOXEL_DIV)
    out_png = (DST_PLOT_ROOT / rel).with_suffix(".png")
    plot_loglog(scales, counts, coeff, out_png, f"{rel.stem}  D≈{D:.3f}")
    print(f"   plot         → {out_png}")
    print(f"   fractal dim  = {D:.3f}")


Random ICs:   0%|          | 0/14 [00:00<?, ?it/s]


→ 1.0578 1081.stl
   cleaned STL  → Edited_STLs\Randomly Generated ICs\1.0578 1081.stl


Random ICs:   7%|▋         | 1/14 [00:04<00:56,  4.33s/it]

   plot         → Boxcount_Plots\Randomly Generated ICs\1.0578 1081.png
   fractal dim  = 2.055

→ 1.2000 1510.stl
   cleaned STL  → Edited_STLs\Randomly Generated ICs\1.2000 1510.stl


Random ICs:  14%|█▍        | 2/14 [00:09<00:56,  4.72s/it]

...

   plot         → Boxcount_Plots\Randomly Generated ICs\1.3423 6050.png
   fractal dim  = 2.367

→ 1.3512 5660.stl
   cleaned STL  → Edited_STLs\Randomly Generated ICs\1.3512 5660.stl


Random ICs:  93%|█████████▎| 13/14 [02:03<00:14, 14.18s/it]

   plot         → Boxcount_Plots\Randomly Generated ICs\1.3512 5660.png
   fractal dim  = 2.355

→ 1.3689 5500.stl
   cleaned STL  → Edited_STLs\Randomly Generated ICs\1.3689 5500.stl


Random ICs: 100%|██████████| 14/14 [02:18<00:00,  9.88s/it]

   plot         → Boxcount_Plots\Randomly Generated ICs\1.3689 5500.png
   fractal dim  = 2.343


In [3]:
# ────────────────────────────────────────────────────────────────────────────
# 3-D Box-Counting  —  RANDOMLY-GENERATED CTs  +  CSV & Summary Plot
# ────────────────────────────────────────────────────────────────────────────

from pathlib import Path
import subprocess, sys, importlib

# ---------- ensure required libs ----------
for pkg in ("trimesh", "numpy", "matplotlib", "tqdm", "pandas"):
    if importlib.util.find_spec(pkg) is None:
        subprocess.check_call([sys.executable, "-m", "pip", "install", pkg, "--quiet"])

import numpy as np, trimesh as tm, pandas as pd
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

# ---------- configuration ----------
SRC_ROOT      = Path("3D_Files") / "Randomly Generated ICs"
DST_STL_ROOT  = Path("Edited_STLs")    / "Randomly Generated ICs"
DST_PLOT_ROOT = Path("Boxcount_Plots") / "Randomly Generated ICs"

TARGET_SIZE   = np.array([30., 30., 36.])   # mm  (X,Y,Z)
PLATE_THICK   = 3.0                         # mm
VOXEL_DIV     = (1, 7)                      # k range for ε = L/2^k

DST_STL_ROOT.mkdir(parents=True, exist_ok=True)
DST_PLOT_ROOT.mkdir(parents=True, exist_ok=True)

# ---------- helpers ----------
def preprocess_mesh(stl_path: Path) -> tm.Trimesh:
    mesh = tm.load_mesh(stl_path, process=False)
    ext  = mesh.bounding_box.extents            # axis-aligned; no SciPy needed
    tall = ext.argmax()
    mesh.apply_scale(TARGET_SIZE[2] / ext[tall])
    if tall != 2:                               # rotate so Z is tall axis
        rot = {0: tm.transformations.rotation_matrix(np.pi/2, [0,1,0]),
               1: tm.transformations.rotation_matrix(-np.pi/2, [1,0,0])}[tall]
        mesh.apply_transform(rot)
    z_min, z_max = mesh.bounds[:, 2]
    for n, off in (([0,0, 1], -(z_min+PLATE_THICK)),
                   ([0,0,-1],  (z_max-PLATE_THICK))):
        mesh = mesh.slice_plane([0,0,-off*n[2]], n)
    mesh.remove_unreferenced_vertices()
    return mesh

def box_count(mesh: tm.Trimesh, k_min, k_max):
    L = mesh.bounding_box.extents.max()
    scales, counts = [], []
    for k in range(k_min, k_max + 1):
        eps  = L / (2**k)
        vox  = mesh.voxelized(eps)
        scales.append(eps)
        counts.append(len(vox.sparse_indices))
    coeff = np.polyfit(np.log(scales), np.log(counts), 1)
    return np.array(scales), np.array(counts), -coeff[0], coeff

def plot_loglog(scales, counts, coeff, png_path, title=""):
    plt.figure(figsize=(4.4,4))
    plt.loglog(scales, counts, "o", label="data")
    plt.loglog(scales,
               np.exp(np.polyval(coeff, np.log(scales))),
               "-", label=f"fit  D={-coeff[0]:.3f}")
    plt.gca().invert_xaxis()
    plt.xlabel("box size ε [mm]")
    plt.ylabel("boxes N(ε)")
    plt.title(title); plt.legend(); plt.tight_layout()
    png_path.parent.mkdir(parents=True, exist_ok=True)
    plt.savefig(png_path, dpi=300); plt.close()

# ---------- main loop ----------
records = []        # will become a DataFrame

for stl in tqdm(list(SRC_ROOT.rglob("*.stl")), desc="Random ICs"):
    rel  = stl.relative_to(SRC_ROOT)
    CID  = stl.stem                          # simple identifier
    print(f"\n→ {rel}")
    mesh = preprocess_mesh(stl)

    out_stl = DST_STL_ROOT / rel
    out_stl.parent.mkdir(parents=True, exist_ok=True)
    mesh.export(out_stl)

    scales, counts, D, coeff = box_count(mesh, *VOXEL_DIV)
    out_png = (DST_PLOT_ROOT / rel).with_suffix(".png")
    plot_loglog(scales, counts, coeff, out_png, f"{CID}  D≈{D:.3f}")

    # ---- collect for summary table ----
    records.append({"CID": CID,
                    "fractal_dimension": round(D, 5),
                    "stl_path": str(out_stl)})

# ---------- summary CSV + overview plot ----------
df = pd.DataFrame(records).sort_values("CID")
df.to_csv("Random_ICs_fractal_dims.csv", index=False)
print("\nSaved summary table → Random_ICs_fractal_dims.csv")

plt.figure(figsize=(max(6, 0.5*len(df)), 4))
plt.bar(df["CID"], df["fractal_dimension"], color="steelblue")
plt.xticks(rotation=90)
plt.ylabel("Minkowski dimension D")
plt.title("Minkowski dimension per Randomly-Generated CT")
plt.tight_layout()
plt.savefig("Random_ICs_fractal_dims.png", dpi=300)
plt.close()
print("Saved overview plot  → Random_ICs_fractal_dims.png")


Random ICs:   0%|          | 0/14 [00:00<?, ?it/s]


→ 1.0578 1081.stl


Random ICs:   7%|▋         | 1/14 [00:03<00:46,  3.54s/it]


→ 1.2000 1510.stl


Random ICs:  14%|█▍        | 2/14 [00:09<00:57,  4.75s/it]

...


→ 1.3512 5660.stl


Random ICs:  93%|█████████▎| 13/14 [02:14<00:14, 14.71s/it]


→ 1.3689 5500.stl


Random ICs: 100%|██████████| 14/14 [02:31<00:00, 10.83s/it]



Saved summary table → Random_ICs_fractal_dims.csv
Saved overview plot  → Random_ICs_fractal_dims.png


In [4]:
# ────────────────────────────────────────────────────────────────────────────
# Random-IC Box-Counting with 3-D box visualisations
# ────────────────────────────────────────────────────────────────────────────
from pathlib import Path
import subprocess, sys, importlib, warnings

# ---------- make sure core deps exist ----------
for pkg in ("trimesh", "numpy", "matplotlib", "tqdm", "pandas"):
    if importlib.util.find_spec(pkg) is None:
        subprocess.check_call([sys.executable, "-m", "pip", "install", pkg, "--quiet"])

import numpy as np, pandas as pd, trimesh as tm
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D      # noqa: F401 (activate 3-D)
from tqdm.auto import tqdm

# ---------- configuration ----------
SRC_ROOT  = Path("3D_Files") / "Randomly Generated ICs"
DST_STL   = Path("Edited_STLs")    / "Randomly Generated ICs"
DST_PLOT  = Path("Boxcount_Plots") / "Randomly Generated ICs"
DST_BOXES = Path("Box_boxes")      / "Randomly Generated ICs"

TARGET_SIZE = np.array([30., 30., 36.])   # mm (X,Y,Z)
PLATE_THICK = 3.0                         # mm
VOXEL_DIV   = (1, 7)                      # k range → ε=L/2^k

for d in (DST_STL, DST_PLOT, DST_BOXES): d.mkdir(parents=True, exist_ok=True)

# ---------- helpers ----------
def preprocess_mesh(stl_path: Path) -> tm.Trimesh:
    mesh  = tm.load_mesh(stl_path, process=False)
    ext   = mesh.bounding_box.extents
    tall  = ext.argmax()
    mesh.apply_scale(TARGET_SIZE[2] / ext[tall])
    if tall != 2:                         # rotate so Z axis is tallest
        rot = {0: tm.transformations.rotation_matrix(np.pi/2, [0,1,0]),
               1: tm.transformations.rotation_matrix(-np.pi/2, [1,0,0])}[tall]
        mesh.apply_transform(rot)
    z0, z1 = mesh.bounds[:, 2]
    for n, off in (([0,0, 1], -(z0+PLATE_THICK)),
                   ([0,0,-1],  (z1-PLATE_THICK))):
        mesh = mesh.slice_plane([0,0,-off*n[2]], n)
    mesh.remove_unreferenced_vertices()
    return mesh

def box_count(mesh: tm.Trimesh, k_min, k_max):
    L = mesh.bounding_box.extents.max()
    scales, counts, grids = [], [], []          # save grids for plotting
    for k in range(k_min, k_max+1):
        eps  = L / (2**k)
        vox  = mesh.voxelized(eps)
        scales.append(eps)
        counts.append(len(vox.sparse_indices))
        grids.append(vox)                       # keep for later view
    coeff = np.polyfit(np.log(scales), np.log(counts), 1)
    return (np.array(scales), np.array(counts),
            -coeff[0], coeff, grids)

def plot_loglog(scales, counts, coeff, png, title=""):
    plt.figure(figsize=(4.5,4))
    plt.loglog(scales, counts, "o", label="data")
    plt.loglog(scales, np.exp(np.polyval(coeff, np.log(scales))),
               "-", label=f"fit  D={-coeff[0]:.3f}")
    plt.gca().invert_xaxis(); plt.xlabel("box size ε [mm]"); plt.ylabel("boxes N(ε)")
    plt.title(title); plt.legend(); plt.tight_layout()
    png.parent.mkdir(parents=True, exist_ok=True); plt.savefig(png, dpi=300); plt.close()

def plot_boxes(vox: tm.voxel.VoxelGrid, out_png):
    """Scatter cube centres to illustrate occupied boxes."""
    pts = vox.points                      # (N,3) centres in mm
    if len(pts) > 50_000:                 # avoid multi-MB PNGs
        pts = pts[np.random.choice(len(pts), 50_000, replace=False)]
    fig = plt.figure(figsize=(5,5))
    ax  = fig.add_subplot(111, projection='3d')
    ax.scatter(pts[:,0], pts[:,1], pts[:,2], s=2, alpha=0.6)
    max_range = (pts.max(0) - pts.min(0)).max()
    mid = pts.mean(0)
    ax.set_xlim(mid[0]-max_range/2, mid[0]+max_range/2)
    ax.set_ylim(mid[1]-max_range/2, mid[1]+max_range/2)
    ax.set_zlim(mid[2]-max_range/2, mid[2]+max_range/2)
    [ax.set_axis_off() for _ in range(3)]
    out_png.parent.mkdir(parents=True, exist_ok=True)
    plt.tight_layout(); plt.savefig(out_png, dpi=300); plt.close()

# ---------- main ----------
records = []
for stl in tqdm(list(SRC_ROOT.rglob("*.stl")), desc="Random ICs"):
    rel  = stl.relative_to(SRC_ROOT);  CID = stl.stem
    print(f"\n→ {rel}")
    mesh = preprocess_mesh(stl)

    # save cleaned STL
    out_mesh = DST_STL / rel;  out_mesh.parent.mkdir(parents=True, exist_ok=True)
    mesh.export(out_mesh)

    # box-count & per-ε visualisations
    scales, counts, D, coeff, grids = box_count(mesh, *VOXEL_DIV)
    # ε-N plot
    plot_loglog(scales, counts, coeff,
                (DST_PLOT / rel).with_suffix(".png"), f"{CID}  D≈{D:.3f}")
    # 3-D box plots
    sub_box_dir = DST_BOXES / CID;  sub_box_dir.mkdir(parents=True, exist_ok=True)
    for k, vox in zip(range(VOXEL_DIV[0], VOXEL_DIV[1]+1), grids):
        img_path = sub_box_dir / f"eps_{k}.png"
        plot_boxes(vox, img_path)

    # collect summary
    records.append({"CID": CID, "fractal_dimension": round(D,5),
                    "stl_path": str(out_mesh)})

# ---------- summary CSV + CID-vs-D plot ----------
df = pd.DataFrame(records).sort_values("CID")
df.to_csv("Random_ICs_fractal_dims.csv", index=False)

plt.figure(figsize=(max(6, 0.5*len(df)), 4))
plt.bar(df["CID"], df["fractal_dimension"], color="steelblue")
plt.xticks(rotation=90); plt.ylabel("Fractal dimension D")
plt.title("Fractal dimension per Randomly-Generated CT")
plt.tight_layout(); plt.savefig("Random_ICs_fractal_dims.png", dpi=300); plt.close()

print("\n✓ Summary table  → Random_ICs_fractal_dims.csv")
print("✓ Overview plot  → Random_ICs_fractal_dims.png")
print("✓ Per-ε box images saved under Box_boxes/Randomly Generated ICs/<CID>/")


Random ICs:   0%|          | 0/14 [00:00<?, ?it/s]


→ 1.0578 1081.stl


Random ICs:   7%|▋         | 1/14 [00:08<01:54,  8.80s/it]


→ 1.2000 1510.stl


Random ICs:  14%|█▍        | 2/14 [00:19<01:58,  9.85s/it]

...


→ 1.3512 5660.stl


Random ICs:  93%|█████████▎| 13/14 [03:58<00:28, 28.33s/it]


→ 1.3689 5500.stl


Random ICs: 100%|██████████| 14/14 [04:40<00:00, 20.02s/it]



✓ Summary table  → Random_ICs_fractal_dims.csv
✓ Overview plot  → Random_ICs_fractal_dims.png
✓ Per-ε box images saved under Box_boxes/Randomly Generated ICs/<CID>/


In [15]:
# ==============================================================
# Conway Trees → GPU voxel pipeline with explicit cube sizes
# ==============================================================

try:
    import cupy as xp; _gpu = True;  print("CuPy on GPU")
except ImportError:
    import numpy as xp; _gpu = False; print("NumPy on CPU")

import numpy as np, pandas as pd, os, math, itertools, shutil
import matplotlib.pyplot as plt; from mpl_toolkits.mplot3d import Axes3D # noqa
from tqdm.auto import tqdm

# ---------------- CONFIG --------------------------------------
N_SAMPLES    = 200
GRID_XY      = 30
NZ_GEN       = 30
CUBE_SIZES = list(range(3, 15))  # Cube sizes from 3 to 14 inclusive
ROOT_OUT     = "CT_CUDA_Output"
if os.path.isdir(ROOT_OUT): shutil.rmtree(ROOT_OUT)
for sub in ("numpy_arrays", "box_plots", "sample_voxels", "summary"):
    os.makedirs(os.path.join(ROOT_OUT, sub), exist_ok=True)

# -------------- GPU Conway helpers ----------------------------
def roll3(a, dx, dy):
    return xp.roll(xp.roll(a, dx, -2), dy, -1)


def conway_step(g):
    """
    One Conway update with DEAD boundary.
    xp is cupy (GPU) or numpy (CPU) depending on _gpu flag.
    """
    p = xp.pad(g, 1)                               # add 1-voxel halo of zeros
    n = (p[:-2, :-2] + p[:-2, 1:-1] + p[:-2, 2:] +   # 8-neighbor sum
         p[1:-1, :-2]              + p[1:-1, 2:] +
         p[2:  , :-2] + p[2:  , 1:-1] + p[2:  , 2:])

    birth   = (g == 0) & (n == 3)
    survive = (g == 1) & ((n == 2) | (n == 3))
    return (birth | survive).astype(xp.uint8)


# -------------- CID (unchanged) -------------------------------
def cid_ratio(bits):
    d={'0':1,'1':2}; code=3; buf=''; tot=0; cur=math.floor(1+math.log2(code))
    for b in bits.astype(str):
        s=buf+b
        if s in d: buf=s
        else:
            tot+=cur; d[s]=code; code+=1
            if code>2**cur: cur+=1
            buf=b
    if buf: tot+=cur
    return tot/len(bits)

# -------------- ε-box count for arbitrary cube sizes ----------
def box_count_grid(vox, sizes):
    """
    Build occupancy grids for every cube size in `sizes`
    and return:
        eps   – np.array of ε = 30 / s
        nums  – np.array of occupied-box counts
        grids – dict{s : Boolean grid}
        D     – Minkowski slope
        coeff – (slope, intercept) of the log-log fit
    """
    L = vox.shape[0]
    eps_list, num_list, grids = [], [], {}
    for s in sizes:
        occ = xp.zeros((s,s,s), bool) if _gpu else np.zeros((s,s,s), bool)
        b   = xp.linspace(0, L, s+1, dtype=int)
        for ix in range(s):
            x0,x1 = b[ix], b[ix+1]
            for iy in range(s):
                y0,y1 = b[iy], b[iy+1]
                for iz in range(s):
                    z0,z1 = b[iz], b[iz+1]
                    occ[ix,iy,iz] = vox[x0:x1,y0:y1,z0:z1].any()
        grids[s] = occ.get() if _gpu else occ
        eps_list.append(L / s)
        num_list.append(int(occ.sum().get() if _gpu else occ.sum()))

    eps  = np.array(eps_list)
    nums = np.array(num_list)
    mask = nums > 0
    coeff = np.polyfit(np.log(eps[mask]), np.log(nums[mask]), 1) if mask.sum()>1 else (np.nan,np.nan)
    D = -coeff[0]
    return eps, nums, grids, round(float(D),4), coeff

def save_fit_plot(eps, nums, coeff, out_png, title=""):
    """log–log scatter + fitted line."""
    plt.figure(figsize=(4,3), dpi=300)
    plt.loglog(eps, nums, "o", label="data")
    fit = np.exp(np.polyval(coeff, np.log(eps)))
    plt.loglog(eps, fit, "-", label=f"fit  D={-coeff[0]:.3f}")
    plt.gca().invert_xaxis()
    plt.xlabel("box size ε"); plt.ylabel("occupied boxes N(ε)")
    plt.title(title, fontsize=8); plt.legend(fontsize=7)
    plt.tight_layout()
    os.makedirs(os.path.dirname(out_png), exist_ok=True)
    plt.savefig(out_png, dpi=300); plt.close()



# -------------- cube PNG util ---------------------------------
def save_cube_plot(grid, path, title=""):
    sx,sy,sz = grid.shape; M=max(sx,sy,sz)
    pad = ((0,M-sx),(0,M-sy),(0,M-sz))
    grid = np.pad(grid, pad) if any(p[1]>0 for p in pad) else grid
    fig = plt.figure(figsize=(4,4)); ax=fig.add_subplot(111,projection='3d')
    ax.voxels(grid, facecolors='teal', edgecolors='k', linewidth=.25)
    ax.set_box_aspect([1,1,1]); ax.set_axis_off(); ax.set_title(title,fontsize=8)
    plt.tight_layout(); os.makedirs(os.path.dirname(path), exist_ok=True)
    plt.savefig(path,dpi=300); plt.close()

# -------------- bubble plot util ------------------------------
def bubble_plot(df,x,y,xlab,ylab,filename,deg):
    sizes=[120*deg[r.Stem] for r in df.itertuples()]
    plt.figure(figsize=(6,3),dpi=300)
    plt.scatter(df[x],df[y],s=sizes,color='k',alpha=.6,edgecolors='none')
    plt.xlabel(xlab); plt.ylabel(ylab)
    plt.gca().spines[['top','right']].set_visible(False)
    plt.tight_layout(); plt.savefig(filename,dpi=300); plt.close()

# ================= MAIN  (η-sweep) ===================================
records, deg = {}, {}        # deg → bubble sizes
rows = []

p_live_values = np.linspace(0.5, 1.0, N_SAMPLES)   # ← uniform sweep

for idx, p_live in tqdm(list(enumerate(p_live_values, 1)),
                        desc="Samples", total=N_SAMPLES):

    eta = p_live                                  # order parameter
    g0  = (xp.random.random((GRID_XY, GRID_XY)) < p_live).astype(xp.uint8)

    # build 30³ voxel tensor
    vox = xp.zeros((GRID_XY, GRID_XY, NZ_GEN), xp.uint8)
    g = g0
    for z in range(NZ_GEN):
        vox[:, :, z] = g
        g = conway_step(g)

    cid    = round(cid_ratio(xp.asnumpy(g0).flatten()), 4)
    vcount = int(vox.sum().get() if _gpu else vox.sum())
    stem   = f"CID{cid:.4f}_eta{eta:.3f}_N{vcount}"

    # ── make sure per-sample box folder exists
    sample_box_dir = f"{ROOT_OUT}/box_plots/{stem}"
    os.makedirs(sample_box_dir, exist_ok=True)

    # ── save tensor & full voxel PNG
    np.save(f"{ROOT_OUT}/numpy_arrays/{stem}.npy", xp.asnumpy(vox))
    save_cube_plot(xp.asnumpy(vox.transpose(2, 1, 0)),
                   f"{ROOT_OUT}/sample_voxels/{stem}.png", stem)

    # ── box-count + ε–box plots + fit plot
    eps, nums, grids, D, coeff = box_count_grid(vox, CUBE_SIZES)
    save_fit_plot(eps, nums, coeff,
                  f"{sample_box_dir}/fit.png", stem)

    for s, occ in grids.items():
        save_cube_plot(occ, f"{sample_box_dir}/eps_{s}.png",
                       f"{stem}  s={s}")

    rows.append(dict(CID=cid, eta=round(eta, 3),
                     VoxelCount=vcount, Stem=stem,
                     Minkowski_D=D))
    deg[stem] = deg.get(stem, 0) + 1

# ---------------- Summary -------------------------------------
df = pd.DataFrame(rows).sort_values("CID").reset_index(drop=True)
df.to_csv(f"{ROOT_OUT}/summary/CID_vs_Minkowski.csv", index=False)

bubble_plot(df, 'CID', 'Minkowski_D',
            "CID", "Minkowski D",
            f"{ROOT_OUT}/summary/CID_vs_D.png", deg)

bubble_plot(df, 'eta', 'Minkowski_D',
            "η", "Minkowski D",
            f"{ROOT_OUT}/summary/eta_vs_D.png", deg)

print("✓ Outputs written to", ROOT_OUT)
df.head()


CuPy on GPU


Samples: 100%|██████████| 200/200 [1:08:26<00:00, 20.53s/it]


✓ Outputs written to CT_CUDA_Output


,CID,eta,VoxelCount,Stem,Minkowski_D
0,0.2222,1.000,904,CID0.2222_eta1.000_N904,2.0
1,0.2556,0.997,901,CID0.2556_eta0.997_N901,2.0
2,0.2689,0.995,899,CID0.2689_eta0.995_N899,2.0
3,0.2756,0.992,899,CID0.2756_eta0.992_N899,2.0
4,0.2889,0.990,897,CID0.2889_eta0.990_N897,2.0


In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import os

# ---------------- Replot CID vs Minkowski D ------------------

# Set paths
summary_csv = "CT_CUDA_Output/summary/CID_vs_Minkowski.csv"
output_png  = "CT_CUDA_Output/summary/CID_vs_D_REPLOT.png"

# Read CSV
df = pd.read_csv(summary_csv)

# Optional: construct dummy deg dictionary for marker size
deg = {r.Stem: 1 for r in df.itertuples()}  # ← uniform size

# Plot
sizes = [45 * deg[r.Stem] for r in df.itertuples()]
plt.figure(figsize=(6, 3), dpi=300)
plt.scatter(df['CID'], df['Minkowski_D'],
            s=sizes, color='k', alpha=0.6, edgecolors='none')

plt.xlabel("CID")
plt.ylabel(r"$\mathrm{dim}_{\mathrm{Minkowski}}(\mathrm{CT})$")
plt.gca().spines[['top', 'right']].set_visible(False)
plt.tight_layout()

# Save figure
os.makedirs(os.path.dirname(output_png), exist_ok=True)
plt.savefig(output_png, dpi=300)
plt.close()

print(f"✓ Plot saved to {output_png}")


✓ Plot saved to CT_CUDA_Output/summary/CID_vs_D_REPLOT.png
